Anexo U

# Despliegue en IBM WATSON CLOUD de función en python que reduce, normaliza, aumenta a 3 canales y devuelve el resultado de la predicción del modelo EfficentNet del anexo S.

#### Elaborado por: Ricardo Niño de Rivera Barrón

#### Ingeniería Biónica

#### Trabajo Terminal II

Debido a que el equipo que se utilizó en el desarrollo del prototipo propuesto en este trabajo es incompatible con bibliotecas de 64 bits, se decidió desplegar la función descrita en esta libreta interactiva en la nube utilizando el servicio de IBM WATSON CLOUD.

Para demostrar la correcta implementación de esta función se comparará su repsuesta en el prototipo final.

**NOTA IMPORTANTE:** En los anexos Q y S la API KEY y la ID de acceso son diferntes a esta libreta porque se implementaron en dierentes cuentas. Esto debido a que en las pruebas se excedi{o la capacidad gratuita permitida por mes en las pruebas. sin embargo, el protoipo final y están realizados en una sola cuenta.

## Configurando el ambiente

Primeramente es necesario crear una instancia gratuita en Watson para utilizar el servicio específico de Machine Learning.

### Conectando a Watson Machine Learning

Para comunicarnos con la instancia creada necesitamos dos datos, "Api Key" y la ubicación geográfica de la instancia.


In [1]:
api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
location = 'us-south'

In [2]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Importando la biblioteca para manipular recursos en IBM Watson

In [3]:
#!pip install -U ibm-watson-machine-learning

In [4]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

2020-12-05 18:19:17,203 - ibm_watson_machine_learning.wml_client_error - WARNING - Failure during scoring. (POST https://us-south.ml.cloud.ibm.com/ml/v4/deployments/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/predictions?version=2020-08-01)
Status code: 403, body: {"trace":"XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX","errors":[{"code":"SPACES0008E","message":"User 'IBMid-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' is not authorized to perform this action on space 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'."}]}
2020-12-05 18:47:12,826 - ibm_watson_machine_learning.wml_client_error - WARNING - Failure during scoring. (POST https://us-south.ml.cloud.ibm.com/ml/v4/deployments/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/predictions?version=2020-08-01)
Status code: 400, body: {"trace": "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX", "errors": [{"code": "score_processing_failure", "message": "local variable 'imagen' referenced before assignment"}], "status_code": 400}



### Trabajando con espacios

Posteriormente se debe establecer un "espacio" de trabajo en en en el servicio de almacenamiento de objetos de IBM Cloud.

Para realizar esta tarea es posible realizarla en esta libreta o directamente en la plataforma.

In [5]:
space_id = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

Enlistando todos los espacios existentes en la instancia creada.

In [6]:
client.spaces.list(limit=10)

------------------------------------  -----------  ------------------------
ID                                    NAME         CREATED
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  WEB_APP_TT2  2020-12-05T17:16:16.303Z
------------------------------------  -----------  ------------------------


Ahora establecemos el espacio creado como espacio por default para desplegar el modelo propuesto.

In [7]:
client.set.default_space(space_id)

'SUCCESS'

In [8]:
client.deployments.get_job_details()

{'resources': []}

In [9]:
!ls

EFNB7_weights.h5  modelo.h5  modelo.tar.gz  T0032.1.1.D.2016-10-29.16.npy


## Descargando ejemplo de prueba

In [10]:
# Instalando biblioteca mega.py
!pip install mega.py

In [11]:
from mega import Mega

In [12]:
mega = Mega()

Inicamos sesión en MEGA con cuenta temporal y anónima

In [13]:
# Log in en la cuenta de MEGA con cuenta temporal anónima
m = mega.login()

In [14]:
# Descargando el archivo de prueba T0032.1.1.D.2016-10-29.16.npy
m.download_url('https://mega.nz/file/URwlCCpC#It8L-P9xJexCtwgFY2q1NbXkEH8A9aVg9JWZfedLLjw')

PosixPath('T0032.1.1.D.2016-10-29.16.npy')

In [15]:
# Mostrando archivos en el directorio de trabajo del entorno
!ls

EFNB7_weights.h5  modelo.h5  modelo.tar.gz  T0032.1.1.D.2016-10-29.16.npy


In [16]:
# Imprimiendo el directorio de trabajo
!pwd

/home/wsuser/work


Leyendo archivo con numpy.

In [17]:
import numpy as np

In [18]:
X_prueba = np.load('T0032.1.1.D.2016-10-29.16.npy')
X_prueba.shape

(480, 640)

## Función a desplegar

Esta función convierte la imagen riginal de resolución 640x420 px a una imagen de 128x128 px y retorna el resultado del modelo desplegado en el anexo S (EfficientNet).

In [19]:
# # importando bibliotecas auxiliares
import tensorflow as tf

Estableciendo el ID de el modelo EfficientNet definido en el Anexo S.

In [60]:
model_deployment_id = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

In [61]:
ai_parms = {"wml_credentials": wml_credentials, "space_id": space_id, "model_deployment_id": model_deployment_id}
def efnb7_function_128x128( parms=ai_parms ):
    
    def get_img_128(imagen, size=128):

        import tensorflow as tf
        import numpy as np
        
        try:
            img = np.expand_dims(imagen, axis=2)
            
            img = tf.image.resize(img, [size,size], antialias=True)/255
            
            img_128 = np.zeros((1, 128, 128,3))
            
            # Almacenamos la imagen obtenida en cada canal
            img_128[0,:,:,0] = img[:,:,0]
            img_128[0,:,:,1] = img[:,:,0]
            img_128[0,:,:,2] = img[:,:,0]
            
            print(img.shape)
            print(img_128.shape)
            
            img_list = img_128.tolist()
            
            return img_list
        
        except Exception as e:
            print(e)
    
        
    def score(function_payload):
        import numpy as np
        try:
            imagen   = np.array(function_payload["input_data"][0]["values"][0])
            X = get_img_128(imagen)
            print(len(X[0]))
            
            model_payload = {"input_data": [{"values": [X[0]]}]}
                    
            from ibm_watson_machine_learning import APIClient
            client       = APIClient(parms["wml_credentials"])
            client.set.default_space(parms["space_id"])
            model_result = client.deployments.score(parms["model_deployment_id"], model_payload)
            digit_class  = model_result["predictions"][0]["values"][0]
                    
            return model_result
        
        except Exception as e:
            return {'predictions': [{'values': [repr(e)]}]}
            #return {"error" : repr(e)}


    return score

## Probando la función localmente

In [62]:
score_0 = X_prueba.tolist()

In [63]:
func_result = efnb7_function_128x128()({"input_data": [{"values": [score_0]}]})
print(func_result)

(128, 128, 1)
(1, 128, 128, 3)
128
{'predictions': [{'id': 'dense_2', 'fields': ['prediction', 'prediction_classes', 'probability'], 'values': [[[0.021548686549067497], [0], [0.021548686549067497]]]}]}


In [64]:
func_result['predictions'][0]['values'][0][0][0]

0.021548686549067497

### Publicando función

Definiendo nombre del modelo, nombre del autor y el software específico.

In [65]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.7")

In [66]:
meta_data = {
    client.repository.FunctionMetaNames.NAME: 'Tranform image to 128x128 function',
    client.repository.FunctionMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

function_details = client.repository.store_function(meta_props=meta_data, function=efnb7_function_128x128)

### Obteniendo detalles del modelo

In [67]:
function_uid = client.repository.get_function_uid(function_details)
function_uid

'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXc'

## Desplegando y almacenando en una nube

Se despliega la función en IBM WATSON CLOUD.

#### Desplegando en la nube la funci+on almacenada (publicada) previamente.

In [68]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Tranform image to 128x128 function deployed",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

function_deployment_details = client.deployments.create(function_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' started

#######################################################################################


initializing..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
------------------------------------------------------------------------------------------------




Ahora podemos imprimir el "online scoring endpoint"

In [69]:
function_deployment_id = client.deployments.get_uid(function_deployment_details)
print(function_deployment_id)

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX


In [70]:
scoring_endpoint = client.deployments.get_scoring_href(function_deployment_details)
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/ml/v4/deployments/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/predictions


Mostrando la lista de despliegues.

In [71]:
client.deployments.list()

------------------------------------  -------------------------------------------  -----  ------------------------
GUID                                  NAME                                         STATE  CREATED
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  Tranform image to 128x128 function deployed  ready  2020-12-05T19:00:13.009Z
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  EfficientNetB7 Deployed                      ready  2020-12-05T17:50:14.274Z
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  Deployment of external Tensorflow model      ready  2020-12-05T17:25:17.606Z
------------------------------------  -------------------------------------------  -----  ------------------------


### Score

Ahora vamos a realizar una prueba de acceso.

In [72]:
scoring_payload = {"input_data": [{"values": [score_0]}]}

Usando el método ``client.deployments.score()``para probar el modelo en la nube.

In [73]:
result = client.deployments.score(function_deployment_id, scoring_payload)
if "error" in result:
    print(result["error"])
else:
    print(result)

{'predictions': [{'id': 'dense_2', 'fields': ['prediction', 'prediction_classes', 'probability'], 'values': [[[0.021548686549067497], [0], [0.021548686549067497]]]}]}


Imprimimos la probabilidad de cáncer. (VALORES ENTRE 0 Y 1)

In [74]:
# Modelo Online
result['predictions'][0]['values'][0][0][0]

0.021548686549067497